# DFL benchmark - training

In [1]:
!nvidia-smi

Sat Oct  8 01:33:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   48C    P8    38W / 350W |    707MiB / 24576MiB |     31%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
# from IPython.display import Video
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.metrics import AUC

# Configuration

In [3]:
DEBUG = False
ALLDATA_USE = False

In [4]:
class CFG:
    # TRAIN_CSV = "/workdir/work/output/3dcnn_valid_images.csv"
    # TRAIN_IMG_DIR = "/workdir/work/output/3dcnn_validdata"
    TRAIN_CSV = "/workdir/work/output/3dcnn_valid_images_allframe.csv"
    TRAIN_IMG_DIR = "/workdir/work/output/3dcnn_validdata_allframe"
    
    IMAGE_SIZE = 256
    NUM_IMAGES = 8
    INPUT_CHANNEL = 3
    BATCH_SIZE = 16
    
    if DEBUG:
        epoch_num = 3
    else:
        epoch_num = 100

## train valid data setting

In [5]:
err_tol = {
    'challenge': [ 0.30, 0.40, 0.50, 0.60, 0.70 ],
    'play': [ 0.15, 0.20, 0.25, 0.30, 0.35 ],
    'throwin': [ 0.15, 0.20, 0.25, 0.30, 0.35 ]
}
validation_video = ['cfbe2e94_0','cfbe2e94_1']
   
if ALLDATA_USE:
    video_id_split = {
        'val':['3c993bd2_0','3c993bd2_1'],
        'train':['1606b0e6_0','1606b0e6_1','35bd9041_0','35bd9041_1',
                 '407c5a9e_1','4ffd5986_0','cfbe2e94_0','cfbe2e94_1',
                 '9a97dae4_1','ecf251d4_0']
    }

event_names = ['challenge', 'throwin', 'play']

# Load Data

In [6]:
train_valid_df = pd.read_csv(CFG.TRAIN_CSV)
display(train_valid_df.head())
df_validation = train_valid_df[train_valid_df["video_id"].isin(validation_video)]

,video_id,frame,file_name
0,cfbe2e94_0,0,/workdir/work/output/3dcnn_validdata_allframe/...
1,cfbe2e94_0,1,/workdir/work/output/3dcnn_validdata_allframe/...
2,cfbe2e94_0,2,/workdir/work/output/3dcnn_validdata_allframe/...
3,cfbe2e94_0,3,/workdir/work/output/3dcnn_validdata_allframe/...
4,cfbe2e94_0,4,/workdir/work/output/3dcnn_validdata_allframe/...


In [7]:
print(len(train_valid_df), len(df_validation))

180413 180413


## Load image check

In [8]:
# (batchsize, width, height, depth, channel) 形式

def load_images_3d(file_names_str_, num_imgs=CFG.NUM_IMAGES, img_size=CFG.IMAGE_SIZE):
    img3d = []
    files = file_names_str_.split(",")
    for file in files:
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (img_size, img_size))
        if len(img3d) > 0:
            # これ転地でいいのか？w,hの方向は要確認
            img3d = np.concatenate([img3d, np.expand_dims(image, 2)], axis=2)
        else:
            img3d = np.expand_dims(image, 2)

    if img3d.shape[2] < num_imgs:
        # n_zero = np.zeros((CFG.INPUT_CHANNEL, img_size, img_size, num_imgs - img3d.shape[2]))
        n_zero = np.zeros((img_size, img_size, num_imgs - img3d.shape[2], CFG.INPUT_CHANNEL))
        # n_zero = np.zeros((num_imgs - img3d.shape[0], img_size, img_size, CFG.INPUT_CHANNEL))
        img3d = np.concatenate((img3d,  n_zero), axis=2)
        
    if np.min(img3d) < np.max(img3d):
        img3d = img3d - np.min(img3d)
        img3d = img3d / np.max(img3d)
            
    return img3d

- データ形式はdataloader で呼び出したときに、(channel, size, size, img_num, batchsize)

input shape = (None, 256, 256, 8, 3)

(batchsize, width, height, depth, channel)

# Dataset

In [9]:
event_decoding = {
    0 : "background",
    1 : "challenge",
    2 : "play",
    3 : "throwin",
}

event_decoding_play = {
    0 : "challenge",
    1 : "play",
    2 : "throwin",
}


# event_encoding = {
#     "background" : 0,
#     "challenge" : 1,
#     "play" : 2,
#     "throwin" : 3,
# }
event_encoding = {
    "background" : [1,0,0,0],
    "challenge" : [0,1,0,0],
    "play" : [0,0,1,0],
    "throwin" : [0,0,0,1],
}

In [10]:
# from keras.utils import Sequence
from tensorflow.keras.utils import Sequence
import math
from random import shuffle

class TestDataset(Sequence):
    def __init__(self, df, num_image=CFG.NUM_IMAGES,batch_size=CFG.BATCH_SIZE):
        self.df = df
        self.idx = df.index.values
        self.video_id = df["video_id"].values
        self.frame = df["frame"].values
        self.file_name = df["file_name"].values
        self.batch_size = batch_size
    
    def __len__(self):
        return int(len(self.df)/self.batch_size)
   
    def __getitem__(self, idx):
        file_path = self.file_name[idx*self.batch_size : (idx+1)*self.batch_size]
        img_3d_list = [load_images_3d(file) for file in file_path]
        input_image_3d = np.stack(img_3d_list, axis=0)

        return input_image_3d

## set dataset

In [11]:
validation_dataset = TestDataset(df_validation, batch_size=CFG.BATCH_SIZE)

# Model

In [12]:
def get_model(width=CFG.IMAGE_SIZE, height=CFG.IMAGE_SIZE, depth=CFG.NUM_IMAGES, channel=CFG.INPUT_CHANNEL):
    """Build a 3D convolutional neural network model."""

    # inputs = keras.Input((width, height, depth, 1))
    inputs = keras.Input((width, height, depth, channel))# rgb?

    x = layers.Conv3D(filters=32, kernel_size=(3,3,3), activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=(2,2,2))(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv3D(filters=32, kernel_size=(3,3,3), activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=(2,2,2))(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv3D(filters=64, kernel_size=(3,3,3), activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=(2,2,2))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.01)(x)
    
    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=1024, activation="relu")(x)
    x = layers.Dropout(0.08)(x)

    outputs = layers.Dense(units=4, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")

    return model

In [13]:
# Build model.
model = get_model()
model.summary()

2022-10-07 12:00:01.935510: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 12:00:01.954727: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 12:00:01.954839: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 8, 3)]  0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 254, 254, 6, 64)   5248      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 127, 127, 3, 64)   0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 127, 127, 3, 64)   256       
_________________________________________________________________
dropout (Dropout)            (None, 127, 127, 3, 64)   0         
_________________________________________________________________
global_average_pooling3d (Gl (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1024)              66560 

2022-10-07 12:00:01.955349: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 12:00:01.955841: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 12:00:01.955929: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 12:00:01.955982: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [14]:
model.load_weights('/workdir/work/output/3dcnn_evendata.h5')

In [15]:
preds = model.predict(validation_dataset)

2022-10-07 12:00:04.982989: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 100663296 exceeds 10% of free system memory.
2022-10-07 12:00:05.070313: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-10-07 12:00:07.040257: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 100663296 exceeds 10% of free system memory.
2022-10-07 12:00:07.581171: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
2022-10-07 12:00:08.809358: W tensorflow/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-10-07 12:00:08.809375: W tensorflow/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at ptxas
2022-10-07 12:00:08.809444: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform

In [16]:
preds.shape

(180400, 4)

# post process function

In [17]:
event_names = ['challenge', 'throwin', 'play']
label_dict = {
    'background':0,
    'challenge':1,
    'play':2,
    'throwin':3,
}
event_names_with_background = ['background','challenge','play','throwin']

def make_sub(prob, pred_df):
    
    frame_rate = 25
    window_size = 128
    ignore_width = 5
    group_count = 10

    df = pd.DataFrame(prob,columns=event_names_with_background)
    df['video_id'] = pred_df['video_id']
    df['frame_id'] = pred_df['frame']

    train_df = pd.DataFrame()
    for video_id, each_video_df in df.groupby('video_id'):
        for i, event in enumerate(event_names):
            # イベント毎にwindow size分の移動平均を取る-> prob_arrに格納(最初と最後のwindow_sizeがたりない分はNanになるので-100で埋める)
            prob_arr = each_video_df[event].rolling(window=window_size, center=True).mean().fillna(-100).values
            each_video_df['rolling_prob'] = prob_arr
            
            sort_arr = np.argsort(-prob_arr)# 全frameの中で、そのフレームのlogitsが何番目に小さいかの順番を格納したarrayを作成
            rank_arr = np.empty_like(sort_arr) # sort_arrと同じshapeの空の配列を作成(実際は空というものはないのでランダムな値が入っている)
            rank_arr[sort_arr] = np.arange(len(sort_arr)) # 各フレームのlogitsが全フレームのうち何番目に小さいかの順番を格納?
            # index list for detected action
            idx_list = []
            for i in range(len(prob_arr)):
                this_idx = sort_arr[i]
                if this_idx >= 0:
                    # Add maximam index to index_list
                    idx_list.append(this_idx)
                    # parityを組んで、こingnorelistを作って、順番が一定以下のものはpredictからはずす(probが高いところの周辺は最高値を残して消えていく)
                    for parity in (-1,1):
                        # 除外対象を考えるために、-1~1のparityに無視する範囲をかけてex_idxを作る
                        for j in range(1, ignore_width+1):
                            ex_idx = this_idx + j * parity
                            # idxがprobの長さ以内にあるときに処理する
                            if ex_idx >= 0 and ex_idx < len(prob_arr):
                                # Exclude frames near this_idx where the action occurred. 
                                sort_arr[rank_arr[ex_idx]] = -1
            this_df = each_video_df.iloc[idx_list].reset_index(drop=True).reset_index().rename(columns={'index':'rank'})[['rank','video_id','frame_id']]
            this_df['event'] = event
            train_df = train_df.append(this_df)  
    
    train_df['time'] = train_df['frame_id']/frame_rate
    train_df['score'] = 1/(train_df['rank']+1)# rankに応じてスコアをつける検出個数が多いほど後ろのscoreは小さくなっていく
    
    return train_df

# make sub

In [18]:
preds_for_sub = preds.copy()

In [19]:
preds_for_sub

array([[0.3603645 , 0.40392736, 0.40949354, 0.42180592],
       [0.35973197, 0.40340242, 0.40904027, 0.42155975],
       [0.35861444, 0.40247393, 0.40827906, 0.42119148],
       ...,
       [0.45418406, 0.34718946, 0.32344294, 0.30859348],
       [0.45512646, 0.3470244 , 0.32336497, 0.3082462 ],
       [0.45632884, 0.34668782, 0.32319662, 0.3076951 ]], dtype=float32)

In [20]:
preds_play = preds_for_sub[:,1:]
print(preds_play)

[[0.40392736 0.40949354 0.42180592]
 [0.40340242 0.40904027 0.42155975]
 [0.40247393 0.40827906 0.42119148]
 ...
 [0.34718946 0.32344294 0.30859348]
 [0.3470244  0.32336497 0.3082462 ]
 [0.34668782 0.32319662 0.3076951 ]]


In [21]:
encode_preds = np.argmax(preds_play, axis=1)
score_plays = np.max(preds_play, axis=1)
pred_plays = [event_decoding_play[pred] for pred in encode_preds]
beforepp_df = df_validation[:len(pred_plays)].copy()
beforepp_df["event"] = pred_plays
beforepp_df["score"] = score_plays

In [22]:
beforepp_df["event"].value_counts()

throwin      91011
challenge    89319
play            70
Name: event, dtype: int64

In [23]:
scoring_df = make_sub(preds_for_sub, df_validation)
display(scoring_df)
scoring_df = scoring_df[['video_id', 'time', 'event', 'score']]
display(scoring_df.head(5))
scoring_df["event"].value_counts()

,rank,video_id,frame_id,event,time,score
0,0,cfbe2e94_0,27538,challenge,1101.52,1.000000
1,1,cfbe2e94_0,27532,challenge,1101.28,0.500000
2,2,cfbe2e94_0,27544,challenge,1101.76,0.333333
3,3,cfbe2e94_0,27526,challenge,1101.04,0.250000
4,4,cfbe2e94_0,27556,challenge,1102.24,0.200000
...,...,...,...,...,...,...
14589,14589,cfbe2e94_1,11,play,0.44,0.000069
14590,14590,cfbe2e94_1,57,play,2.28,0.000069
14591,14591,cfbe2e94_1,51,play,2.04,0.000069
14592,14592,cfbe2e94_1,45,play,1.80,0.000069


,video_id,time,event,score
0,cfbe2e94_0,1101.52,challenge,1.000000
1,cfbe2e94_0,1101.28,challenge,0.500000
2,cfbe2e94_0,1101.76,challenge,0.333333
3,cfbe2e94_0,1101.04,challenge,0.250000
4,cfbe2e94_0,1102.24,challenge,0.200000


throwin      29565
play         29335
challenge    29289
Name: event, dtype: int64

In [24]:
solution = pd.read_csv("/workdir/work/input/train.csv", usecols=['video_id', 'time', 'event'])
scoring_solution = solution[solution['video_id'].isin(validation_video)]

# Scoring

In [25]:
# copy from https://www.kaggle.com/code/ryanholbrook/competition-metric-dfl-event-detection-ap

import numpy as np
import pandas as pd
from pandas.testing import assert_index_equal
from typing import Dict, Tuple

tolerances = {
    "challenge": [0.3, 0.4, 0.5, 0.6, 0.7],
    "play": [0.15, 0.20, 0.25, 0.30, 0.35],
    "throwin": [0.15, 0.20, 0.25, 0.30, 0.35],
}

def filter_detections(
        detections: pd.DataFrame, intervals: pd.DataFrame
) -> pd.DataFrame:
    """Drop detections not inside a scoring interval."""
    detection_time = detections.loc[:, 'time'].sort_values().to_numpy()
    intervals = intervals.to_numpy()
    is_scored = np.full_like(detection_time, False, dtype=bool)

    i, j = 0, 0
    while i < len(detection_time) and j < len(intervals):
        time = detection_time[i]
        int_ = intervals[j]

        # If the detection is prior in time to the interval, go to the next detection.
        if time < int_.left:
            i += 1
        # If the detection is inside the interval, keep it and go to the next detection.        
        elif time in int_:
            is_scored[i] = True
            i += 1
        # If the detection is later in time, go to the next interval.
        else:
            j += 1

    return detections.loc[is_scored].reset_index(drop=True)


def match_detections(
        tolerance: float, ground_truths: pd.DataFrame, detections: pd.DataFrame
) -> pd.DataFrame:
    """Match detections to ground truth events. Arguments are taken from a common event x tolerance x video evaluation group."""
    detections_sorted = detections.sort_values('score', ascending=False).dropna()

    is_matched = np.full_like(detections_sorted['event'], False, dtype=bool)
    gts_matched = set()
    for i, det in enumerate(detections_sorted.itertuples(index=False)):
        best_error = tolerance
        best_gt = None

        for gt in ground_truths.itertuples(index=False):
            error = abs(det.time - gt.time)
            if error < best_error and not gt in gts_matched:
                best_gt = gt
                best_error = error
            
        if best_gt is not None:
            is_matched[i] = True
            gts_matched.add(best_gt)

    detections_sorted['matched'] = is_matched

    return detections_sorted


def precision_recall_curve(
        matches: np.ndarray, scores: np.ndarray, p: int
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    if len(matches) == 0:
        return [1], [0], []

    # Sort matches by decreasing confidence
    idxs = np.argsort(scores, kind='stable')[::-1]
    scores = scores[idxs]
    matches = matches[idxs]
    
    distinct_value_indices = np.where(np.diff(scores))[0]
    threshold_idxs = np.r_[distinct_value_indices, matches.size - 1]
    thresholds = scores[threshold_idxs]
    
    # Matches become TPs and non-matches FPs as confidence threshold decreases
    tps = np.cumsum(matches)[threshold_idxs]
    fps = np.cumsum(~matches)[threshold_idxs]
    
    precision = tps / (tps + fps)
    precision[np.isnan(precision)] = 0
    recall = tps / p  # total number of ground truths might be different than total number of matches
    
    # Stop when full recall attained and reverse the outputs so recall is non-increasing.
    last_ind = tps.searchsorted(tps[-1])
    sl = slice(last_ind, None, -1)

    # Final precision is 1 and final recall is 0
    return np.r_[precision[sl], 1], np.r_[recall[sl], 0], thresholds[sl]


def average_precision_score(matches: np.ndarray, scores: np.ndarray, p: int) -> float:
    precision, recall, _ = precision_recall_curve(matches, scores, p)
    # Compute step integral
    return -np.sum(np.diff(recall) * np.array(precision)[:-1])


def event_detection_ap(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        tolerances: Dict[str, float],
) -> float:

    assert_index_equal(solution.columns, pd.Index(['video_id', 'time', 'event']))
    assert_index_equal(submission.columns, pd.Index(['video_id', 'time', 'event', 'score']))

    # Ensure solution and submission are sorted properly
    solution = solution.sort_values(['video_id', 'time'])
    submission = submission.sort_values(['video_id', 'time'])
    
    # Extract scoring intervals.
    intervals = (
        solution
        .query("event in ['start', 'end']")
        .assign(interval=lambda x: x.groupby(['video_id', 'event']).cumcount())
        .pivot(index='interval', columns=['video_id', 'event'], values='time')
        .stack('video_id')
        .swaplevel()
        .sort_index()
        .loc[:, ['start', 'end']]
        .apply(lambda x: pd.Interval(*x, closed='both'), axis=1)
    )

    # Extract ground-truth events.
    ground_truths = (
        solution
        .query("event not in ['start', 'end']")
        .reset_index(drop=True)
    )

    # Map each event class to its prevalence (needed for recall calculation)
    class_counts = ground_truths.value_counts('event').to_dict()

    # Create table for detections with a column indicating a match to a ground-truth event
    detections = submission.assign(matched = False)

    # Remove detections outside of scoring intervals
    detections_filtered = []
    for (det_group, dets), (int_group, ints) in zip(
        detections.groupby('video_id'), intervals.groupby('video_id')
    ):
        assert det_group == int_group
        detections_filtered.append(filter_detections(dets, ints))
    detections_filtered = pd.concat(detections_filtered, ignore_index=True)

    # Create table of event-class x tolerance x video_id values
    aggregation_keys = pd.DataFrame(
        [(ev, tol, vid)
         for ev in tolerances.keys()
         for tol in tolerances[ev]
         for vid in ground_truths['video_id'].unique()],
        columns=['event', 'tolerance', 'video_id'],
    )

    # Create match evaluation groups: event-class x tolerance x video_id
    detections_grouped = (
        aggregation_keys
        .merge(detections_filtered, on=['event', 'video_id'], how='left')
        .groupby(['event', 'tolerance', 'video_id'])
    )
    ground_truths_grouped = (
        aggregation_keys
        .merge(ground_truths, on=['event', 'video_id'], how='left')
        .groupby(['event', 'tolerance', 'video_id'])
    )
    
    # Match detections to ground truth events by evaluation group
    detections_matched = []
    for key in aggregation_keys.itertuples(index=False):
        dets = detections_grouped.get_group(key)
        gts = ground_truths_grouped.get_group(key)
        detections_matched.append(
            match_detections(dets['tolerance'].iloc[0], gts, dets)
        )
    detections_matched = pd.concat(detections_matched)
    
    # Compute AP per event x tolerance group
    event_classes = ground_truths['event'].unique()
    ap_table = (
        detections_matched
        .query("event in @event_classes")
        .groupby(['event', 'tolerance']).apply(
        lambda group: average_precision_score(
        group['matched'].to_numpy(),
                group['score'].to_numpy(),
                class_counts[group['event'].iat[0]],
            )
        )
    )

    # Average over tolerances, then over event classes
    mean_ap = ap_table.groupby('event').mean().mean()

    return mean_ap

# Score

In [26]:
scoring_df = scoring_df.reset_index(drop=True)
display(scoring_df)

,video_id,time,event,score
0,cfbe2e94_0,1101.52,challenge,1.000000
1,cfbe2e94_0,1101.28,challenge,0.500000
2,cfbe2e94_0,1101.76,challenge,0.333333
3,cfbe2e94_0,1101.04,challenge,0.250000
4,cfbe2e94_0,1102.24,challenge,0.200000
...,...,...,...,...
88184,cfbe2e94_1,0.44,play,0.000069
88185,cfbe2e94_1,2.28,play,0.000069
88186,cfbe2e94_1,2.04,play,0.000069
88187,cfbe2e94_1,1.80,play,0.000069


In [27]:
scoring_solution = scoring_solution.reset_index(drop=True)
display(scoring_solution)

,video_id,time,event
0,cfbe2e94_0,229.321518,start
1,cfbe2e94_0,230.200000,play
2,cfbe2e94_0,232.520000,play
3,cfbe2e94_0,234.016200,end
4,cfbe2e94_0,246.666301,start
...,...,...,...
1581,cfbe2e94_1,3562.660000,play
1582,cfbe2e94_1,3563.835896,end
1583,cfbe2e94_1,3572.500727,start
1584,cfbe2e94_1,3574.340000,throwin


In [28]:
score = event_detection_ap(scoring_solution, scoring_df, tolerances)
print(score)

0.036826990804473124


In [29]:
beforepp_df["time"] = beforepp_df["frame"]/25
score = event_detection_ap(scoring_solution, beforepp_df[['video_id', 'time', 'event', 'score']], tolerances)
print(score)

0.0014880731676787252


- inteval:0.32   ,  0.036649694269400436,  0.04085720712321188